# Process MGRS Squares

In [ ]:
# need to upgrade odc-stac to fix keyerror issue for some tiles
!pip install -q odc-stac -U

In [ ]:
import numpy as np
import pandas as pd
import geopandas as gpd
import rasterio as rio
import pystac
import pystac_client
import stackstac
import matplotlib.pyplot as plt
import xarray as xr
from datetime import datetime
import matplotlib
import ipyleaflet
import sys
import os
import dask_gateway
import planetary_computer
from rechunker import rechunk
sys.path.append('../sar_snowmelt_timing')
import s1_rtc_bs_utils
import contextily as ctx
import rioxarray as rxr
import skimage
import pathlib
import glob
import re
import time
import fsspec

In [ ]:
cluster = dask_gateway.GatewayCluster()
client = cluster.get_client()
cluster.adapt(minimum=100, maximum=400)
print(client.dashboard_link)

In [ ]:
url = 'https://github.com/scottyhq/mgrs/raw/main/MGRS_LAND.parquet' # Scott created an MGRS parquet file here https://github.com/scottyhq/mgrs
with fsspec.open(url) as file:
    mgrs_gdf = gpd.read_parquet(file)

In [ ]:
tile_names_10T = list(mgrs_gdf[mgrs_gdf.tile.str.match('10T[DEFG]')].tile)

In [ ]:
tile_names = open('../input/MGRS_square_list/square_list.txt','r').readlines()
tile_names = [i.rstrip('\n') for i in tile_names]

In [ ]:
tile_names = [tile for tile in tile_names if tile in tile_names_10T]

In [ ]:
years = [2015,2016,2017,2018,2019,2020,2021,2022]

In [ ]:
resolution = 40

In [ ]:
classes = [ # page 13 of https://esa-worldcover.s3.amazonaws.com/v100/2020/docs/WorldCover_PUM_V1.0.pdf
#    10, # treecover
    20, # shrubland
    30, # grassland
    40, # cropland
#    50, # built-up
    60, #bare / sparse vegetation
    70, # snow and ice
#    80, # permanent water bodies
    90, # herbaceous wetlands
    95, # mangroves
    100 # loss and lichen
]

In [ ]:
for nm in tile_names:
    print(f'Processing {nm}:')
    bbox_gdf = mgrs_gdf[mgrs_gdf.tile==nm]
    ts_ds = s1_rtc_bs_utils.get_s1_rtc_stac_odc_pc(bbox_gdf,start_time='2014-01-01',end_time='2022-12-31',resolution=resolution)
    worldcover = s1_rtc_bs_utils.get_worldcover(ts_ds)
    ts_ds = ts_ds.where(worldcover.isin(classes))
    
    path = f'../output/MGRS/{nm}' 
    savedir = pathlib.Path(path)
    savedir.mkdir(parents=True,exist_ok=True)
    
    for year in years:
        print(f'Processing {year}...')
        tic = time.perf_counter()
        one_year = slice(f'{year}-01-01',f'{year}-12-31')
        ts_ds_oneyear = ts_ds.sel(time=one_year) 
        # does this rechunking step help time
        #ts_ds_oneyear = ts_ds_oneyear.chunk((100,1,512,512))
        runoffs_median = s1_rtc_bs_utils.get_runoff_onset(ts_ds_oneyear)
        runoffs_median_computed = runoffs_median.compute()
        # add write crs step
        runoffs_median_computed.rio.write_crs(rio.CRS.from_epsg(bbox_gdf['epsg']),inplace=True)
        runoffs_median_computed.dt.dayofyear.rio.to_raster(f'../output/MGRS/{nm}/runoff_onset_{nm}_{year}_median_{resolution}m.tif')
        toc = time.perf_counter()
        print(f'Processing time: {toc - tic:0.1f} seconds')
print(f'Done!')

# Now loop through all folders, create an all year median

In [ ]:
for nm in tile_names:
    print(f'Processing {nm}:')
    geotiff_list = glob.glob(f'../output/MGRS/{nm}/runoff_onset_*.tif')
    
    year_list = []
    for geotiff in geotiff_list:
        year_list.append(re.search("([0-9]{4})", geotiff).group(0))
    year_list = [int(year) for year in year_list]

     #Create variable used for time axis
    time_var = xr.Variable('time', year_list)

    # Load in and concatenate all individual GeoTIFFs
    runoffs_allyears = xr.concat([rxr.open_rasterio(i) for i in geotiff_list],dim=time_var).squeeze().sortby('time')
    
    #runoffs_allyears.plot(col='time')
    runoffs_allyears.median(dim='time').rio.to_raster(f'../output/MGRS/{nm}/runoff_onset_{nm}_allyears_median_{resolution}m.tif')
    runoffs_allyears.std(dim='time').rio.to_raster(f'../output/MGRS/{nm}/runoff_onset_{nm}_allyears_std_{resolution}m.tif')
    polyfits= runoffs_allyears.polyfit('time',deg=1,full=True,cov=True)
    polyfits.polyfit_coefficients[0].rio.set_crs(runoffs_allyears.rio.crs).rio.to_raster(f'../output/MGRS/{nm}/runoff_onset_{nm}_allyears_polyfit_{resolution}m.tif')
    xr.corr(runoffs_allyears.time,runoffs_allyears,dim='time').rio.set_crs(runoffs_allyears.rio.crs).rio.to_raster(f'../output/MGRS/{nm}/runoff_onset_{nm}_allyears_corr_strength_{resolution}m.tif')